## Flow to extract the main topic from a sentence.

References:

> [Structured Entity Extraction Using Large Language Models](https://arxiv.org/pdf/2402.04437v3)

> [Simple topic extraction](https://medium.com/analytics-vidhya/fun-with-topic-extraction-8aa11e0437d0)

> [Effective sentence-level relation extraction model using entity-centric dependency tree](https://peerj.com/articles/cs-2311/)

> [Universal sentence encoder based intent representation](https://medium.com/webknossos/enhancing-intent-classification-with-the-universal-sentence-encoder-ecbcd7a3005c)

In [3]:
# Install required packages for first run. 
# %pip install tensorflow_hub #Uncomment to install if needed.

In [7]:
#Import necessary packages.
import tensorflow_hub as hub
import tensorflow as tf

In [8]:
# Example sentences (Just some random news headlines)
sentences = [
    "Air India crash LIVE | 'On highest alert, all rescue teams mobilised,' says Civil Aviation Minister",
    "Israel Gave Patriot Missiles To Ukraine? Bibi's 'Panic Call' To Putin After Diplomat's Big Reveal",
    "Epic Games Files Lawsuit Against Fortnite Cheat Developer."
]

In [ ]:
#Simple topic extraction


In [ ]:
#universal encoder based:

# Load the Universal Sentence Encoder model
model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(model_url)

# Generate embeddings
embeddings = embed(sentences)

# Display embeddings
for i, sentence in enumerate(sentences):
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embeddings[i][:5]}...")  # Display first 5 dimensions for brevity
